In [0]:
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
from databricks.vector_search.client import VectorSearchClient

In [0]:
%sql
CREATE TABLE IF NOT EXISTS healthcare_lifesciences.qsar.zinc15_250k_full CLONE hls_drug_bank.qsar.zinc15_250k_full;

In [0]:
#endpoint_name="zinc_vs"
endpoint_name="dbdemos_vs_endpoint"
vs_index="healthcare_lifesciences.qsar.zinc_vs"
source_table = "healthcare_lifesciences.qsar.zinc15_250k_full"

In [0]:
client = VectorSearchClient()

In [0]:
try:    
    client.create_endpoint(name=endpoint_name)
except Exception as e:
    if 'ALREADY_EXISTS' in str(e):
        print(f"Endpoint {endpoint_name} already exists.")
    else:
        raise

In [0]:
index = client.create_delta_sync_index(
  endpoint_name=endpoint_name,
  source_table_name=source_table,
  index_name=vs_index,
  columns_to_sync=["smiles", "zinc_id", "mwt", "logp", 
                   "reactive", "purchasable", "tranche_name", "qed", "SPS", "TPSA", "FractionCSP3", "RingCount", "ecfp"],
  pipeline_type="TRIGGERED",
  primary_key="zinc_id",
  embedding_dimension=1024,
  embedding_vector_column="ecfp"
)